In [ ]:
%%capture
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !git clone https://github.com/ivanbrillo/tesiRFX
    %cd tesiRFX
    !rsync -a --exclude='*.ipynb' --exclude='*.git' ./ ../
    %cd ..
    !rm -rf tesiRFX
    
import tensorflow as tf
from tensorflow.keras import layers
import customLib.autoencoder_helper as helper
from customLib.AE import AE, get_sequentials_outer

In [ ]:
train, test = helper.get_splitted_np(win_size=80, alpha=40)

In [ ]:
autoencoder = AE(*get_sequentials_outer())
autoencoder.set_trainable(False)
autoencoder.build(input_shape=(None, 1800, 1))
autoencoder.load_weights("./weights/convAE.h5")

In [ ]:
def get_seq_fullAE():
    encoder = tf.keras.Sequential([
        layers.Input(shape=(1800, 1)),
        autoencoder.encoder,
        layers.Dense(70, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(30, activation='relu'),
        layers.Dense(10, activation='relu'),
        layers.Dense(2, activation='linear'),
    ])

    decoder = tf.keras.Sequential([
        layers.Input(shape=(2,)),
        layers.Dense(10, activation='linear'),
        layers.Dense(30, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(70, activation='relu'),
        layers.Dense(80, activation='linear'),
        autoencoder.decoder,
    ])

    return encoder, decoder


full_autoencoder = AE(*get_seq_fullAE())

In [ ]:
helper.train_and_evaluate(full_autoencoder, train, test, epochs_n=5000, apply_filter=True, show_latent=True, patience=400)